In [1]:
# # Installations required:
# pip install mygene
# pip install unipressed

In [2]:
import mygene
from unipressed import UniprotkbClient # Access Uniprot

In [3]:
gene_list = ['mtor', 'atg12', 'uvrag', 'ap1m1', 'bcl2']
ensembl_ids = ['ENSG00000198793', 'ENSG00000145782', 'ENSG00000198382', 'ENSG00000072958', 'ENSG00000171791']

### Step 1: use mygene to get the codes for all potential isoforms for the given gene symbol

*Note* this is kind of like going to the gene's [Uniprot page](https://www.uniprot.org/uniprotkb/P42345/entry) and scrolling down to the 'Computationally mapped potential isoform sequences' section.

In [4]:
def find_isoforms(gene_list):
    """
    Given a list of gene symbols, returns a dictionary containing the Uniprot IDs for all computationally predicted isoforms for those genes.
    
    Uses mygene to find all potential isoforms associated with a specific gene symbol.

    """
    # Make the caller object
    mg = mygene.MyGeneInfo()

    # Settings of your choice
    scopes = 'symbol' # replace this with ensembl.gene if you have the ensembl gene id instead
    # fields = 'uniprot'
    fields = 'all'
    species = 'human'
    verbose = False

    # Query the databse
    results = mg.querymany(gene_list, 
            scopes= scopes, 
            fields= fields, 
            species=species, 
            verbose=verbose)
    
    print(results)
    
    # Make a dictionary from the results, getting the most salient info
    isoforms = {i['query']:i['uniprot']['TrEMBL'] for i in results}
    
    return isoforms, results

iso, results = find_isoforms(gene_list)
results

[{'query': 'mtor', 'AllianceGenome': '3942', 'HGNC': '3942', 'MIM': '601231', '_id': '2475', '_score': 17.53251, 'accession': {'genomic': ['AF152840.1', 'AJ300188.1', 'AL031291.3', 'AL049653.7', 'AL109811.40', 'AL359082.16', 'AL391561.20', 'CH471130.1', 'CP068277.2', 'JB157964.1', 'KF495866.1', 'KJ399980.1', 'KT584057.1', 'NC_000001.11', 'NC_060925.1', 'NG_033239.1'], 'protein': ['AAA58486.1', 'AAC39933.1', 'AAC41713.1', 'AAI17167.1', 'BAE06077.2', 'BAG54371.1', 'BAG64047.1', 'BAG65134.1', 'CAC15570.1', 'CAC42395.1', 'CCW44092.1', 'EAW71681.1', 'EAW71682.1', 'EAW71683.1', 'NP_001373429.1', 'NP_001373430.1', 'NP_004949.1', 'P42345.1', 'XP_011539468.1', 'XP_016856389.1', 'XP_047272677.1', 'XP_047272680.1', 'XP_054191719.1', 'XP_054191720.1', 'XP_054191721.1', 'XP_054191722.1'], 'rna': ['AA725390.1', 'AB209995.1', 'AK126762.1', 'AK302863.1', 'AK304273.1', 'BC117166.1', 'BP250183.1', 'DC403129.1', 'L34075.1', 'L35478.1', 'NM_001386500.1', 'NM_001386501.1', 'NM_004958.4', 'U88966.1', 'XM_01

[{'query': 'mtor',
  'AllianceGenome': '3942',
  'HGNC': '3942',
  'MIM': '601231',
  '_id': '2475',
  '_score': 17.53251,
  'accession': {'genomic': ['AF152840.1',
    'AJ300188.1',
    'AL031291.3',
    'AL049653.7',
    'AL109811.40',
    'AL359082.16',
    'AL391561.20',
    'CH471130.1',
    'CP068277.2',
    'JB157964.1',
    'KF495866.1',
    'KJ399980.1',
    'KT584057.1',
    'NC_000001.11',
    'NC_060925.1',
    'NG_033239.1'],
   'protein': ['AAA58486.1',
    'AAC39933.1',
    'AAC41713.1',
    'AAI17167.1',
    'BAE06077.2',
    'BAG54371.1',
    'BAG64047.1',
    'BAG65134.1',
    'CAC15570.1',
    'CAC42395.1',
    'CCW44092.1',
    'EAW71681.1',
    'EAW71682.1',
    'EAW71683.1',
    'NP_001373429.1',
    'NP_001373430.1',
    'NP_004949.1',
    'P42345.1',
    'XP_011539468.1',
    'XP_016856389.1',
    'XP_047272677.1',
    'XP_047272680.1',
    'XP_054191719.1',
    'XP_054191720.1',
    'XP_054191721.1',
    'XP_054191722.1'],
   'rna': ['AA725390.1',
    'AB209995

In [13]:
results[0]['gsrs.protein']

KeyError: 'gsrs.protein'

### Step 2: Use the Uniprot client to get the sequences of each isoform identified in (1)

*Note* this is equivalent to clicking on the identifiers on the uniprot page mentioned above.

In [5]:
def get_uniprot_record(uniprot_record):
    """
    Returns the protein sequence associated with the uniprot record, in homo sapiens.
    """
    records = UniprotkbClient.search(
        query=f"({uniprot_record}) AND (organism_id:9606)"    # 9606 is homo sapiens
    ).each_record()
    record = next(records)
    return {uniprot_record: record['sequence']['value']}

We can do the lookup in batches as well:

In [6]:
def get_uniprot_records(records_list):
    """
    Given a list of uniprot ids, returns a dictionary with associated protein sequences.
    """
    return {i['primaryAccession']:i['sequence']['value'] for i in UniprotkbClient.fetch_many(records_list)}

get_uniprot_records(iso['mtor'])

{'A0A8V8TQP2': 'MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQHYVTMELREMSQEESTRFYDQLNHHIFELVSSSDANERKGGILAIASLIGVEGGNATRIGRFANYLRNLLPSNDPVVMEMASKAIGRLAMAGDTFTAEYVEFEVKRALEWLGADRNEGRRHAAVLVLRELAISVPTFFFQQVQPFFDNIFVAVWDPKQAIREGAVAALRACLILTTQREPKEMQKPQWYRHTFEEAEKGFDETLAKEKGMNRDDRIHGALLILNELVRISSMEGERLREEMEEITQQQLVHDKYCKDLMGFGTKPRHITPFTSFQAVQPQQSNALVGLLGYSSHQGLMGFGTSPSPAKSTLVESRCCRDLMEEKFDQVCQWVLKCRNSKNSLIQMTILNLLPRLAAFRPSAFTDTQYLQDTMNHVLSCVKKEKERTAAFQALGLLSVAVRSEFKVYLPRVLDIIRAALPPKDFAHKRQKAMQVDATVFTCISMLARAMGPGIQQDIKELLEPMLAVGLSPALTAVLYDLSRQIPQLKKDIQDGLLKMLSLVLMHKPLRHPGMPKGLAHQLASPGLTTLPEASDVGSITLALRTLGSFEFEDPDIRYCVLASLDERFDAHLAQAENLQALFVALNDQVFEIRELAICTVGRLSSMNPAFVMPFLRKMLIQILTELEHSGIGRIKEQSARMLGHLVSNAPRLIRPYMEPILKALILKLKDPDPDPNPGVINNVLATIGELAQVSGLEMRKWVDELFIIIMDMLQDSSLLAKRQVALWTLGQLVASTGYVVEPYRKYPTLLEVLLNFLKTEQNQGTRREAIRVLGLLGALDPYKHKVNIGMIDQSRDASAVSLSESKSSQDSSDYSTSEMLVNMGNLPLDEFYPAVSMVALMRIFRDQSLSHHHTMVVQAITFIFKSLGLKCVQFLPQVMPTFLNVIRVCDGAIREFLFQQLGMLVSFVKSHIRPYMDEIVTLMRVRILGHEHLNSEHDHSS

You can pass just a single item in a list as well:

In [7]:
get_uniprot_records([iso['mtor'][0]])

{'A0A8V8TQP2': 'MLGTGPAAATTAATTSSNVSVLQQFASGLKSRNEETRAKAAKELQHYVTMELREMSQEESTRFYDQLNHHIFELVSSSDANERKGGILAIASLIGVEGGNATRIGRFANYLRNLLPSNDPVVMEMASKAIGRLAMAGDTFTAEYVEFEVKRALEWLGADRNEGRRHAAVLVLRELAISVPTFFFQQVQPFFDNIFVAVWDPKQAIREGAVAALRACLILTTQREPKEMQKPQWYRHTFEEAEKGFDETLAKEKGMNRDDRIHGALLILNELVRISSMEGERLREEMEEITQQQLVHDKYCKDLMGFGTKPRHITPFTSFQAVQPQQSNALVGLLGYSSHQGLMGFGTSPSPAKSTLVESRCCRDLMEEKFDQVCQWVLKCRNSKNSLIQMTILNLLPRLAAFRPSAFTDTQYLQDTMNHVLSCVKKEKERTAAFQALGLLSVAVRSEFKVYLPRVLDIIRAALPPKDFAHKRQKAMQVDATVFTCISMLARAMGPGIQQDIKELLEPMLAVGLSPALTAVLYDLSRQIPQLKKDIQDGLLKMLSLVLMHKPLRHPGMPKGLAHQLASPGLTTLPEASDVGSITLALRTLGSFEFEDPDIRYCVLASLDERFDAHLAQAENLQALFVALNDQVFEIRELAICTVGRLSSMNPAFVMPFLRKMLIQILTELEHSGIGRIKEQSARMLGHLVSNAPRLIRPYMEPILKALILKLKDPDPDPNPGVINNVLATIGELAQVSGLEMRKWVDELFIIIMDMLQDSSLLAKRQVALWTLGQLVASTGYVVEPYRKYPTLLEVLLNFLKTEQNQGTRREAIRVLGLLGALDPYKHKVNIGMIDQSRDASAVSLSESKSSQDSSDYSTSEMLVNMGNLPLDEFYPAVSMVALMRIFRDQSLSHHHTMVVQAITFIFKSLGLKCVQFLPQVMPTFLNVIRVCDGAIREFLFQQLGMLVSFVKSHIRPYMDEIVTLMRVRILGHEHLNSEHDHSS

### Bonus: a way of mapping between different identifiers in mygene

In [40]:
def geneID1_to_geneID2(gene_list, scopes = 'symbol', fields = 'ensembl.gene', species = 'human', verbose = False, delimiter = ','):
    """ 
    Given a gene list of a particular identifier type, converts to the id specified in 'fields'.

    Tested with known genes ('TP53', 'MDM2', 'AP1S1') for:
        'symbol' -> 'ensembl.gene'
        'symbol' -> 'ensembl.transcript'
        'ensembl.transcript' -> 'symbol'
        'ensembl.gene' -> 'symbol'
    """
    # Make sure we don't query with an empty list
    if gene_list[0] == '':
        return 'NaN'

    mg = mygene.MyGeneInfo()
    
    results = mg.querymany(gene_list, 
        scopes= scopes, 
        fields= fields, 
        species=species, 
        verbose=verbose)

    field_map = {
        'symbol' : ['symbol'],
        'ensembl.gene': ['ensembl','gene'],
        'ensembl.transcript': ['ensembl','transcript'],
        'refseq.genomic': ['refseq', 'genomic'],
        'refseq.mRNA': ['refseq', 'rna'],
        'refseq.protein': ['refseq', 'protein'],
    }

    ids_map = {}

    for result in results:
        try:
            add_to_dict(ids_map, key = result['query'], value = get_subdictionary_item(result, field_map[fields]))
        except:
            add_to_dict(ids_map, key = result['query'], value = 'NaN')

    # Turn any lists (in values) into one long string
    for k, v in ids_map.items():
        ids_map[k] = condense_list(v)

        # # We should get a list. If there's just one item in the list, extract and return it
        # if len(v) == 1:
        #     ids_map[k] = condense_list(v)
        # else:
        #     # Join each item in the list into a string, then return the string
        #     v = condense_list(v)
        #     ids_map[k] = v

    return ids_map